In [6]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

# Load the CSV data
try:
    df = pd.read_csv('gyroscope_data.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert timestamp to datetime
except FileNotFoundError:
    print("Error: 'gyroscope_data.csv' not found. Please check the file path.")
    df = pd.DataFrame({'timestamp': [], 'x': [], 'y': [], 'z': []})

app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Gyroscope Data Visualization'),
    
    html.Div([
        html.Label('Select Graph Type:'),
        dcc.Dropdown(
            id='graph-type',
            options=[
                {'label': 'Line Chart', 'value': 'line'},
                {'label': '3D Scatter', 'value': '3d_scatter'},
                {'label': 'Distribution Plot', 'value': 'distribution'}
            ],
            value='line'
        ),
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label('Select Axis:'),
        dcc.Dropdown(
            id='axis-select',
            options=[
                {'label': 'X', 'value': 'x'},
                {'label': 'Y', 'value': 'y'},
                {'label': 'Z', 'value': 'z'},
                {'label': 'All', 'value': 'all'}
            ],
            value='all'
        ),
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label('Number of samples to display:'),
        dcc.Input(id='n-samples', type='number', value=100),
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Button('Previous', id='prev-button'),
    html.Button('Next', id='next-button'),
    
    dcc.Graph(id='data-graph'),
    
    dash_table.DataTable(
        id='summary-table',
        columns=[{"name": i, "id": i} for i in ["Statistic", "X", "Y", "Z"]],
        data=[],
        style_table={'width': '50%'}
    )
])

@app.callback(
    [Output('data-graph', 'figure'),
     Output('summary-table', 'data')],
    [Input('graph-type', 'value'),
     Input('axis-select', 'value'),
     Input('n-samples', 'value'),
     Input('prev-button', 'n_clicks'),
     Input('next-button', 'n_clicks')],
    [State('data-graph', 'figure')]
)
def update_graph(graph_type, axis, n_samples, prev_clicks, next_clicks, current_figure):
    ctx = dash.callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else None
    
    start_index = 0
    if current_figure and 'layout' in current_figure and 'xaxis' in current_figure['layout']:
        current_range = current_figure['layout']['xaxis'].get('range', [0, n_samples])
        start_index = df.index[df['timestamp'] >= pd.to_datetime(current_range[0])].min()
    
    if button_id == 'prev-button' and start_index > 0:
        start_index = max(0, start_index - n_samples)
    elif button_id == 'next-button' and start_index + n_samples < len(df):
        start_index = start_index + n_samples
    
    end_index = min(start_index + n_samples, len(df))
    df_subset = df.iloc[start_index:end_index]
    
    fig = go.Figure()
    
    if graph_type == 'line':
        if axis == 'all':
            fig = px.line(df_subset, x='timestamp', y=['x', 'y', 'z'])
        else:
            fig = px.line(df_subset, x='timestamp', y=axis)
    elif graph_type == '3d_scatter':
        fig = px.scatter_3d(df_subset, x='x', y='y', z='z')
    elif graph_type == 'distribution':
        if axis == 'all':
            fig = go.Figure()
            for col in ['x', 'y', 'z']:
                fig.add_trace(go.Histogram(x=df_subset[col], name=col))
            fig.update_layout(barmode='overlay')
        else:
            fig = px.histogram(df_subset, x=axis)
    
    fig.update_layout(title=f'Gyroscope Data: {graph_type.capitalize()}')
    
    # Calculate summary statistics
    summary_data = [
        {"Statistic": stat, "X": df_subset['x'].agg(stat), 
         "Y": df_subset['y'].agg(stat), "Z": df_subset['z'].agg(stat)}
        for stat in ['mean', 'median', 'std', 'min', 'max']
    ]
    
    return fig, summary_data

if __name__ == '__main__':
    app.run_server(debug=True, port = 8000)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[6], line 91, in update_graph(
    graph_type='distribution',
    axis='all',
    n_samples=None,
    prev_clicks=None,
    next_clicks=None,
    current_figure={'data': [{'name': 'x', 'type': 'histogram', 'x': [-0.24]}, {'name': 'y', 'type': 'histogram', 'x': [-0.79]}, {'name': 'z', 'type': 'histogram', 'x': [-0.43]}], 'layout': {'barmode': 'overlay', 'template': {'data': {'bar': [{'error_x': {...}, 'error_y': {...}, 'marker': {...}, 'type': 'bar'}], 'barpolar': [{'marker': {...}, 'type': 'barpolar'}], 'carpet': [{'aaxis': {...}, 'baxis': {...}, 'type': 'carpet'}], 'choropleth': [{'colorbar': {...}, 'type': 'choropleth'}], 'contour': [{'colorbar': {...}, 'colorscale': [...], 'type': 'contour'}], 'contourcarpet': [{'colorbar': {...}, 'type': 'contourcarpet'}], 'heatmap': [{'colorbar': {...}, 'colorscale': [...], 'type': 'heatmap'